# 5 eda aeda back

### 1) Set functions
- ["Okt", "Kkma", "Komoran", "Mecab", "Hannanum"]

In [1]:
# # 분할을 원하지 않는 단어 모아두기
# no_tokenized = ['#Person#', '#Person1#', '#Person2#', '#Person3#', 
# '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#사람1#',
# '#PhoneNumber#', '#Address#', '#PassportNumber#',
# '#DateOfBirth#','#SSN#','#CardNumber#','#CarNumber#','#Email#']

In [2]:
# from jamo import h2j, j2hcj

# def get_jongsung_TF(sample_text):
#     sample_text_list = list(sample_text)
#     last_word = sample_text_list[-1]
#     last_word_jamo_list = list(j2hcj(h2j(last_word)))
#     last_jamo = last_word_jamo_list[-1]

#     jongsung_TF = "T"

#     if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
#         jongsung_TF = "F"

#     return jongsung_TF
    

# # tmp 경로에 저장된 사용자 사전 명사에 새로운 단어 추가
# with open('../mecab-0.996-ko-0.9.2/mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv', "r", encoding='utf-8') as f:
#   user_dict = f.readlines()

# for word in no_tokenized:
#   jongsung_TF = get_jongsung_TF(word)
#   line = '{},*,*,*,NNP,*,{},{},*,*,*,*,*\n'.format(word, jongsung_TF, word)

#   user_dict.append(line)
  
# print(user_dict[-10:])

In [3]:
# # 재저장
# with open('../mecab-0.996-ko-0.9.2/mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv', 'w', encoding='utf-8') as f:
#   for line in user_dict:
#     f.write(line)

In [4]:
# load library
from koeda import EasyDataAugmentation, AEasierDataAugmentation
import pandas as pd
from tqdm.notebook import tqdm
import random
import time

# EDA
def augment_text_data_with_EDA(text_col, repetition):
    # 단어 바꾸기
    """입력된 문장에 대해서 EDA를 통해 데이터 증강"""
    eda = EasyDataAugmentation(
        morpheme_analyzer="Mecab"
        )
    
    random.seed(42)

    result = text_col.progress_apply(eda, p=(0.8, 0.01, 0.02, 0.01), repetition=repetition)
    print(result)
    
    return result

# AEDA
def augment_text_data_with_AEDA(text_col, repetition):
    # 단어 사이에 문자 추가하기
    """입력된 문장에 대해서 AEDA를 통해 데이터 증강"""
    aeda = AEasierDataAugmentation(
        morpheme_analyzer="Mecab",
        punctuations=[".", ",", "!", "?", ";", ":"]
    )

    random.seed(42)
    
    # result = text_col.progress_apply(aeda, p=0.05, repetition=repetition)
    result = []
    for i in tqdm(range(len(text_col))):
        time.sleep(0.01)
        a = aeda(text_col[i], p = 0.05, repetition=repetition)
        result.append(a)

    print(result)
    return result

In [5]:
# pandas tqdm 호출
tqdm.pandas() # tqdm의 pandas전용 메소드를 호출합니다.

### 2) application train data

In [6]:
# load original data
train = pd.read_csv('../data/train.csv')

In [7]:
# eda
train2 = train.copy()
train2['dialogue'] = augment_text_data_with_EDA(train2['dialogue'], 1)
train2.to_csv('../data/mecab_eda_augmentation.csv', index = False)

  0%|          | 0/12457 [00:00<?, ?it/s]

0        #Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...
1        #Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?#Person2#: ...
2        #Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?#Person2#: 어떤 종...
3        #Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?#Person2#...
4        #Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...
                               ...                        
12452    #Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?#Person2#:...
12453    #Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 4시에 도착해야 한다고 했죠?...
12454    #Person1#: 오늘 어떻게 도와드릴까요?#Person2#: 차를 빌리고 싶습니...
12455    #Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?#Person2#:...
12456    #Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...
Name: dialogue, Length: 12457, dtype: object


In [8]:
# # aeda
# train3 = train.copy()
# train3['dialogue'] = augment_text_data_with_AEDA(train3['dialogue'], 1)
# train3.to_csv('../data/mecab_aeda_augmentation.csv')

In [14]:
# 자동화 안되서 분할 적용하기 위해 수동으로 적용
aeda = AEasierDataAugmentation(
    morpheme_analyzer="Mecab",
    punctuations=[".", ",", "!", "?", ";", ":"]
)

random.seed(42)

# result = text_col.progress_apply(aeda, p=0.05, repetition=repetition)
result = []
n = len(train['dialogue'])
for i in tqdm(range(3000)):
    a = aeda(train['dialogue'][i], p = 0.05, repetition=1)
    result.append(a)

  0%|          | 0/3000 [00:00<?, ?it/s]

In [15]:
# 2탄
for i in tqdm(range(3000, 6000)):
    a = aeda(train['dialogue'][i], p = 0.05, repetition=1)
    result.append(a)

  0%|          | 0/3000 [00:00<?, ?it/s]

In [19]:
# 3탄
for i in tqdm(range(7368, 9000)):
    a = aeda(train['dialogue'][i], p = 0.05, repetition=1)
    result.append(a)

  0%|          | 0/1632 [00:00<?, ?it/s]

In [20]:
# 4탄
for i in tqdm(range(9000, n)):
    a = aeda(train['dialogue'][i], p = 0.05, repetition=1)
    result.append(a)

  0%|          | 0/3457 [00:00<?, ?it/s]

In [21]:
# 결과 비교
len(result)

12457

In [22]:
# 동일한 개수 확인
len(train['dialogue'])

12457

In [23]:
# 저장
train3 = train.copy()
train3['dialogue'] = result
train3.to_csv('../data/mecab_aeda_augmentation.csv', index = False)

In [24]:
# 통합 후 저장
train_merge = pd.concat([train, train2, train3])
train_merge.to_csv('../data/augment_train_merge.csv', index = False)